In [1]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout



In [2]:
# used a dictionary to represent an intents JSON file
with open("../training_data/intents.json", "r", encoding='utf-8') as f:
    data = json.load(f)
    
print(data)

{'intents': [{'tag': 'greetings', 'patterns': ['Hi', 'Hey', 'Hello', 'How are you?', 'Good day'], 'responses': ["Hello, I'm the University Chatbot"], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context_set': ''}, {'tag': 'closing', 'patterns': ['Bye', 'Goodbye'], 'responses': ['Goodbye'], 'context_set': ''}, {'tag': 'email', 'patterns': ['What is the email address of Finance Office?', 'Finance Office email address', 'Email address Finance Office', 'Finance email', 'Email Finance'], 'responses': ['The email address of the Finance Office is<br><a>finance.ar@slu.edu.ph</a> <br> For online payments please email your reciept on this email: <br> <a>claims.payment.tuition@slu.edu.ph</a> <br> <a>finance.ar@slu.edu.ph</a><br> <a>claimspayment@slu.edu.ph</a>'], 'context_set': ''}, {'tag': 'landline', 'patterns': ['What are the SLU landline numbers?', 'SLU landline numb

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))
print(words)

["'s", 'a', 'about', 'ac', 'accountancy', 'address', 'an', 'and', 'applicant', 'application', 'applying', 'arch', 'archi', 'architecture', 'are', 'art', 'available', 'b', 'ba', 'beced', 'beed', 'bio', 'biology', 'bpe', 'bsarchi', 'bsba', 'bsba-em', 'bsba-hrdm', 'bsce', 'bsche', 'bscs', 'bsed', 'bsee', 'bsem', 'bsge', 'bshm', 'bsie', 'bsit', 'bsme', 'bsmls', 'bspolsci', 'bsswk', 'by', 'bye', 'can', 'checklist', 'com', 'computing', 'contact', 'could', 'course', 'curiculim', 'curiculum', 'day', 'dean', 'description', 'do', 'doe', 'education', 'email', 'eng', 'engineer', 'engineering', 'enginering', 'engl', 'enroll', 'enrollment', 'entrance', 'entrep', 'exam', 'examination', 'existing', 'failed', 'fee', 'finance', 'finman', 'for', 'freshies', 'freshman', 'good', 'goodbye', 'grade', 'have', 'hello', 'helpful', 'hey', 'hi', 'hotline', 'how', 'i', 'if', 'in', 'incoming', 'industrial', 'information', 'is', 'it', 'it/cs', 'landline', 'law', 'liberal', 'lot', 'ma', 'management', 'math', 'me', 'm

In [4]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [5]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               23168     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 39)                2535      
Total params: 33,959
Trainable params: 33,959
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
7/7 [==============================] - 0s 1ms/step - loss: 3.5438 - accuracy: 0.0433
Epoch 2/200
7/7 

7/7 [==============================] - 0s 1ms/step - loss: 0.1065 - accuracy: 0.9615
Epoch 75/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2052 - accuracy: 0.9471
Epoch 76/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1525 - accuracy: 0.9423
Epoch 77/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1640 - accuracy: 0.9375
Epoch 78/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1399 - accuracy: 0.9567
Epoch 79/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1606 - accuracy: 0.9519
Epoch 80/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1628 - accuracy: 0.9375
Epoch 81/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1993 - accuracy: 0.9567
Epoch 82/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1185 - accuracy: 0.9712
Epoch 83/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1693 - accuracy: 0.9375
Epoch 84/200
7/7 [===============

7/7 [==============================] - 0s 1ms/step - loss: 0.0753 - accuracy: 0.9712
Epoch 158/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1283 - accuracy: 0.9423
Epoch 159/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0926 - accuracy: 0.9663
Epoch 160/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1004 - accuracy: 0.9760
Epoch 161/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1272 - accuracy: 0.9423
Epoch 162/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1433 - accuracy: 0.9519
Epoch 163/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1915 - accuracy: 0.9375
Epoch 164/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1347 - accuracy: 0.9567
Epoch 165/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2585 - accuracy: 0.9471
Epoch 166/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1722 - accuracy: 0.9423
Epoch 167/200
7/7 [=====

In [6]:
context = {}

def clean_text(text): 
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab): 
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    return np.array(bow)

def pred_class(text): 
    bow = bag_of_words(text, words)
    results = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(results) if res > thresh]

    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def get_response(text, userID='123', show_details=True): 
    results = pred_class(text)
    while results:
        for i in data["intents"]: 
            if i["tag"] == results[0][0]:
#                 print(i)
                if "context_set" in i:
                    if show_details: print ('context:', i['context_set'])
                    context[userID] = i["context_set"]

                if not "context_filter" in i or (userID in context and "context_filter" in i and i["context_filter"] == context[userID]):
                    if show_details: print ('tag:', i['tag'])
                    return random.choice(i["responses"])
                
                else:
                    return "Sorry, I can't understand your query."

        results.pop(0)
        

In [ ]:
from flask import Flask, render_template, request, redirect, url_for 

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")
    
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return str(get_response(userText))

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2021 15:17:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:17:29] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2021 15:17:31] "GET /get?msg=no HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 15:17:32] "GET /get?msg=yes HTTP/1.1" 200 -
